In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

In [3]:
# set up the driver
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome("/chromedriver/chromedriver")

date = ""
region = ""
cj1 = ""
cj2 = ""
cj3 = ""
srj1 = ""
srj2 = "" 
srj3 = ""
meta_info = [date, region, cj1, cj2, cj3, srj1, srj2, srj3]

# Navigate to the page
def home():
    driver.get("https://www.texasmusicforms.com/csrrptUILpublic.asp")

# get results button function
def click_results_button():
    get_results_button = driver.find_element(by=By.CLASS_NAME, value="btn-success")
    get_results_button.click()


# "Search again" button function
def click_search_button():
    search_again_button = driver.find_element(by=By.NAME, value="chan")
    search_again_button.click()
    driver.implicitly_wait(10)

# find the drop down menus and fill them out
def form_completer(year, region, contest_item):
    year_menu = driver.find_element(by=By.NAME, value="yr")
    region_menu = driver.find_element(by=By.NAME, value="reg")
    event_menu = driver.find_element(by=By.NAME, value="ev")
    year_menu.send_keys(year)
    region_menu.send_keys(f"Region {region}")
    event_menu.send_keys("Band")

    # find the hidden drop down menu, and collect its contents
    contest_menu = driver.find_element(by=By.NAME, value="cn")
    contest_menu_contents = contest_menu.find_elements(by=By.TAG_NAME, value="option")
    contest_menu_length = len(contest_menu_contents)
    contest_menu.send_keys(contest_menu_contents[contest_item].text)

# scrape the data
def scraper_loop():
    # make sure that were seeing all results, not just the top 20
    show_all_button = driver.find_element(by=By.NAME, value="DataTables_Table_0_length")
    show_all_button.send_keys("All")
    # get the table
    table = driver.find_elements(by=By.TAG_NAME, value="tbody")[1]
    table_rows = table.find_elements(by=By.TAG_NAME, value="tr")

    judges_table = driver.find_elements(by=By.TAG_NAME, value="tbody")[0]
    judges_table_rows = judges_table.find_elements(by=By.TAG_NAME, value="tr")
    date = judges_table_rows[3].find_elements(by=By.TAG_NAME, value="td")[0].text
    region = judges_table_rows[3].find_elements(by=By.TAG_NAME, value="td")[3].text
    cj1 = judges_table_rows[4].find_elements(by=By.TAG_NAME, value="td")[1].text
    cj2 = judges_table_rows[5].find_elements(by=By.TAG_NAME, value="td")[1].text
    cj3 = judges_table_rows[6].find_elements(by=By.TAG_NAME, value="td")[1].text
    srj1 = judges_table_rows[4].find_elements(by=By.TAG_NAME, value="td")[3].text
    srj2 = judges_table_rows[5].find_elements(by=By.TAG_NAME, value="td")[3].text
    srj3 = judges_table_rows[6].find_elements(by=By.TAG_NAME, value="td")[3].text
    #print(date, region, cj1, cj2, cj3, srj1, srj2, srj3)

    # loop through table contents
    for row in table_rows:
        cells = row.find_elements(by=By.TAG_NAME, value="td")
        column_content = []
        for cell in cells:
            cell_content_full = cell.text
            cell_content = cell_content_full.split("\n")
            for i in cell_content:
                column_content.append(i)
        column_content.append(date)
        column_content.append(region)
        column_content.append(cj1)
        column_content.append(cj2)
        column_content.append(cj3)
        column_content.append(srj1)
        column_content.append(srj2)
        column_content.append(srj3)
        row_content.append(column_content)
        

# convert to dataframe
def convert_to_dataframe():
    df = pd.DataFrame(row_content)
    if len(df.columns) == 21:
        df.columns = columns_21
    elif len(df.columns) == 22:
        df.columns = columns_22
    # export to csv
    df.to_csv(f"full_run/texas_music_forms_{year}_{region}_{contest_item}_band.csv")

columns_21 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3']

columns_22 = ['Event', 'School', 'TEA', 'City', 'Directors', 'Conference', 'Classification', 'Year', 'ID', 
    'Stage Judge 1', 'Stage Judge 2', 'Stage Judge 3', 'Stage Final', 
    'SR Judge 1', 'SR Judge 2', 'SR Judge 3', 'SR Final', 'Award', 
    'Selection 1', 'Selection 2', 'Selection 3', 'oops']

row_content = []

# Define year and region variables
year = 2005
region = 1

for y in range(2005, 2023):
    for r in range(0, 33):
        try:
            contest_item = 1
            region = region + 1
            for i in range(1, 15):
                try:
                    home()
                    form_completer(year, region, contest_item)
                    click_results_button()
                    scraper_loop()
                    convert_to_dataframe()
                    contest_item = contest_item + 1
                    row_content = []
                except:
                    print(f"{year}, {region}, {contest_item}")
                    break
        except:
            break
    year = year + 1
    region = 0

2005, 2, 9
2005, 3, 2
2005, 4, 1
2005, 5, 1
2005, 6, 1
2005, 7, 1
2005, 8, 1
2005, 9, 1
2005, 10, 1
2005, 11, 1
2005, 12, 1
2005, 13, 1
2005, 14, 1
2005, 15, 1
2005, 16, 1
2005, 17, 1
2005, 18, 1
2005, 19, 1
2005, 20, 1
2005, 21, 1
2005, 22, 1
2005, 23, 1
2005, 24, 1
2005, 25, 1
2005, 26, 1
2005, 27, 1
2005, 28, 1
2005, 29, 1
2005, 30, 1
2005, 31, 1
2005, 32, 1
2005, 33, 1
2005, 34, 1
2006, 1, 1
2006, 2, 1
2006, 3, 1
2006, 4, 1
2006, 5, 1
2006, 6, 1
2006, 7, 1
2006, 8, 1
2006, 9, 1
2006, 10, 1
2006, 11, 1
2006, 12, 1
2006, 13, 1
2006, 14, 1
2006, 15, 1
2006, 16, 1
2006, 17, 1
2006, 18, 1
2006, 19, 1
2006, 20, 1
2006, 21, 1
2006, 22, 1
2006, 23, 1
2006, 24, 1
2006, 25, 1
2006, 26, 1
2006, 27, 1
2006, 28, 1
2006, 29, 1
2006, 30, 1
2006, 31, 1
2006, 32, 1
2006, 33, 1
2007, 1, 1
2007, 2, 1
2007, 3, 1
2007, 4, 1
2007, 5, 1
2007, 6, 1
2007, 7, 1
2007, 8, 1
2007, 9, 1
2007, 10, 1
2007, 11, 1
2007, 12, 1
2007, 13, 1
2007, 14, 1
2007, 15, 1
2007, 16, 1
2007, 17, 1
2007, 18, 1
2007, 19, 1
2007, 